# Sentiment Analysis of orders reviews in Greek 
### Out-of-core-Learning 

In [1]:
#Import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.corpus import stopwords

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

file_path = "/kaggle/input/skroutz-shop-reviews-sentiment-analysis/Skroutz_dataset.xlsx"

/kaggle/input/skroutz-shop-reviews-sentiment-analysis/Skroutz_dataset.xlsx


In [42]:
df = pd.read_excel(file_path)
df.head()

,id,review,label
0,0,Πολύ καλό κατάστημα τόσο από κοντά όσο και...,Positive
1,1,Γρήγορη παραλαβή και φιλική εξυπηρέτηση.,Positive
2,2,Εύκολα γρήγορα απλά. Το προϊόν όπως περιγρά...,Positive
3,3,Πολύ ευχαριστημένη με την παράδοση. Επικοινω...,Positive
4,4,Όλα πήγαν καλά με την παραγγελία μου. Θα αγο...,Positive


In [31]:
df.isna().sum()

id        0
review    0
label     0
dtype: int64

In [43]:
df.drop(['id'], axis=1, inplace=True)
df.head()

,review,label
0,Πολύ καλό κατάστημα τόσο από κοντά όσο και...,Positive
1,Γρήγορη παραλαβή και φιλική εξυπηρέτηση.,Positive
2,Εύκολα γρήγορα απλά. Το προϊόν όπως περιγρά...,Positive
3,Πολύ ευχαριστημένη με την παράδοση. Επικοινω...,Positive
4,Όλα πήγαν καλά με την παραγγελία μου. Θα αγο...,Positive


In [38]:
df.tail()

,id,review,label
6547,6547,Παράγγειλα κινητό 1 έως 3 εργάσιμες ήθελα ...,Negative
6548,6548,Έχει επιλογές για έκδοση τιμολογίου. Σε αφήν...,Negative
6549,6549,Η παραγγελία ήταν ένα Huawei p10 lite gold e...,Negative
6550,6550,Καθυστέρησαν να μου στείλουν την παραγγελία....,Negative
6551,6551,Δεν ξέρω ακόμη αν θα μου στείλουν το προϊόν!...,Negative


In [44]:
df['label'].replace({'Negative':0, 'Positive':1}, inplace=True)
df.head()

/tmp/ipykernel_30/2419573970.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].replace({'Negative':0, 'Positive':1}, inplace=True)
/tmp/ipykernel_30/2419573970.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'].replace({'Negative':0, 'Positive':1}, inplace=True)


,review,label
0,Πολύ καλό κατάστημα τόσο από κοντά όσο και...,1
1,Γρήγορη παραλαβή και φιλική εξυπηρέτηση.,1
2,Εύκολα γρήγορα απλά. Το προϊόν όπως περιγρά...,1
3,Πολύ ευχαριστημένη με την παράδοση. Επικοινω...,1
4,Όλα πήγαν καλά με την παραγγελία μου. Θα αγο...,1


In [45]:
df.to_csv('Skroutz_dataset.csv')

In [46]:
data = pd.read_csv('/kaggle/working/Skroutz_dataset.csv')
data.head()

,Unnamed: 0,review,label
0,0,Πολύ καλό κατάστημα τόσο από κοντά όσο και...,1
1,1,Γρήγορη παραλαβή και φιλική εξυπηρέτηση.,1
2,2,Εύκολα γρήγορα απλά. Το προϊόν όπως περιγρά...,1
3,3,Πολύ ευχαριστημένη με την παράδοση. Επικοινω...,1
4,4,Όλα πήγαν καλά με την παραγγελία μου. Θα αγο...,1


In [47]:
greek_stopwords = stopwords.words('greek')

In [56]:
# remove any html characters 
def tokenizer(text): 
    text = re.sub('<[^>]*>', '', text)
    emotions = re.findall('(?::|;|=)(?:-)?(?:\)|\)|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emotions).replace('-','')
    tokenized = [w for w in text.split() if w not in greek_stopwords]
    return tokenized

In [57]:
# Generate stream_docs
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [58]:
# Testing the function stream_docs
next(stream_docs(path='/kaggle/working/Skroutz_dataset.csv'))

('0,  Πολύ καλό κατάστημα  τόσο από κοντά  όσο και εξ αποστάσεως. Θα ήταν καλό  όμως  όταν λέμε 1-3 εργάσιμες για την παράδοση να διευκρινίζουμε σε εμφανές σημείο ότι δεν μετράνε από την ημέρα της παραγγελίας  αλλά από την ημέρα της τελικής επεξεργασίας της παραγγελίας  οπότε να μην λέμε 1-3 αλλά 2 ως 4 ή οτιδήποτε άλλο ισχύει. Ο πελάτης βλέπει 1 ως 3 και καταλαβαίνει ότι το αργότερο την τρίτη εργάσιμη από την παραγγελία θα λάβει το προϊόν. Δεν είναι υποχρεωμένος να γνωρίζει τις διαδικασίες της εταιρείας  ούτε τον αφορούν. Αν χρειάζεται μια ημέρα (ή 2 ...) για επεξεργασία  ας συνυπολογισθεί στην περιγραφή της παράδοσης του προϊόντος και όχι στα ψιλά γράμματα.     ',
 1)

# Get mini batches

In [59]:
# Get mini batches
def get_minibatch(doc_stream, size):
    docs, y =[], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

# Training the Model

In [60]:
# Training the Model
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [61]:
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)


In [62]:
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

In [63]:
doc_stream = stream_docs(path='/kaggle/working/Skroutz_dataset.csv')

In [67]:
#!pip install pyprind
#import pyprind
#pbar = pyprind.ProgBar(45)

In [68]:
classes = np.array([0, 1])

In [69]:
progress = ""
for _ in range(6):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break;
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    # pbar.update()
    progress += "#"
    print(progress)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


#
##
###
####
#####
######


In [70]:
# Test with the rest 552 data
X_test, y_test = get_minibatch(doc_stream, size=552)
X_test = vect.transform(X_test)

In [72]:
print('Accuracy: %.3f' % clf.score(X_test, y_test)) 

Accuracy: 0.953


In [73]:
clf = clf.partial_fit(X_test, y_test)

In [74]:
print('Accuracy: %.3f' % clf.score(X_test, y_test)) 

Accuracy: 0.998


# Testing the Model 

### Example 1

In [75]:
# Test 
import pandas as pd
import numpy as np
label = {0: 'negative', 1:'positive'}
example = ['Πολύ καλό κατάστημα  τόσο από κοντά  όσο και εξ αποστάσεως.']
X = vect.transform(example)

In [76]:
# Make a prediction 
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X)) * 100))

Prediction: positive
Probability: 62.75%


### Example 2

In [77]:
example = ['Μου αρέσει αυτό το κατάστημα γιατί είναι το χειρότερο από όλα με χάλια εξυπηρέτηση']
X = vect.transform(example)
# Make a prediction 
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X)) * 100))

Prediction: negative
Probability: 90.19%


# Save the Model

In [78]:
## Save the Model 
import pickle
import os

In [79]:
dest = os.path.join('reviewsclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

In [80]:
pickle.dump(greek_stopwords, open(os.path.join(dest, 'greek_stopwords.pkl'), 'wb'), protocol=4)

In [81]:
pickle.dump(clf, open(os.path.join(dest,'classifier.pkl'), 'wb'), protocol=4)

# Test the saved model

In [82]:
# Test the saved model
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os 
import pickle
import numpy as np

In [85]:
files_dir = '/kaggle/working/reviewsclassifier'
stop = pickle.load(open(os.path.join(files_dir, 'pkl_objects', 'greek_stopwords.pkl'), 'rb'))

In [86]:
def tokenizer(text): 
    text = re.sub('<[^>]*>', '', text)
    emotions = re.findall('(?::|;|=)(?:-)?(?:\)|\)|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emotions).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [87]:
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)

In [88]:
clf = pickle.load(open(os.path.join(files_dir, 'pkl_objects', 'classifier.pkl'), 'rb'))

### Example 1 with Positive review

In [89]:
label = {0:'negative',1:'positive'}
example = ['Πολύ καλή εξυπηρέτηση και γρήγορη παράδοση']
X = vect.transform(example)

In [90]:
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X)) * 100))

Prediction: positive
Probability: 98.84%


### Example 2 with Negative review

In [91]:
label = {0:'negative',1:'positive'}
example = ['το προϊόν δεν ήρθε ποτέ και δεν μας επιστράφηκαν τα χρήματα']
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X)) * 100))

Prediction: negative
Probability: 95.32%


In [1]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: C:\Users\Antonis\anaconda3\Lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: imbalanced-learn
